#Scrape SEC web pages for 10-Ks

In [0]:
from bs4 import BeautifulSoup

import urllib.request
import urllib
import re

In [0]:
def get_all_text_from_url(url_to_scrape):
  page_html = urllib.request.urlopen(url_to_scrape)
  page_parse =  BeautifulSoup(page_html, 'html.parser')
  return page_parse.text

In [0]:
# sec_base_url= "https://www.sec.gov/cgi-bin/browse-edgar?CIK=aapl&type=10-K&owner=exclude&action=getcompany&Find=Search"
sec_base_url_prefix = "https://www.sec.gov/cgi-bin/browse-edgar?CIK="
sec_base_url_suffix= "&type=10-K&owner=exclude&action=getcompany&Find=Search"
SEC_main_url = "https://www.sec.gov"


def get_SEC_base_url_from_stock_ticker(ticker):
  sec_base_url = sec_base_url_prefix + ticker + sec_base_url_suffix
  return sec_base_url


def get_all_10k_text_from_ticker(stock_ticker):
  list_of_10k_text = []
  sec_base_url = get_SEC_base_url_from_stock_ticker(stock_ticker)
  results_page = urllib.request.urlopen(sec_base_url)
  SEC_page_parse = BeautifulSoup(results_page, 'html.parser')

  # the third table is the one that contains the filed documents for various years
  company_data_table = SEC_page_parse.findAll('table')[2]
  for row in company_data_table.findAll('tr'):
    for link in row.findAll('a', attrs={'href': re.compile("/Archives/edgar")}):
      link_text = link.get('href')
      documents_page_url = SEC_main_url+link_text

      doc_page_pre_parse = urllib.request.urlopen(documents_page_url)
      doc_page_parse = BeautifulSoup(doc_page_pre_parse, 'html.parser')
      doc_table = doc_page_parse.find('table', attrs={'summary' : "Document Format Files"})

      # get the first hyperlink in the table
      doc_link = doc_table.find('a')
      doc_link_text = doc_link.get('href')
      doc_link_text = SEC_main_url + doc_link_text

      last_four_chaaracters_of_filename = doc_link_text[-4:]
      if last_four_chaaracters_of_filename == ".txt" or last_four_chaaracters_of_filename == ".htm":
        print('getting 10K text from',doc_link_text)

        text_of_10k = get_all_text_from_url(doc_link_text)
        list_of_10k_text.append(text_of_10k)
  return list_of_10k_text

stock_ticker = "msft"
all_10k_text = get_all_10k_text_from_ticker(stock_ticker)




In [0]:
from wordcloud import WordCloud, STOPWORDS
import matplotlib.pyplot as plt

# Creating a word cloud from a string


Create a word cloud from the list of strings that represent all 10-Ks.

In [0]:
allText = ""
num = 0
for one10K in all_10k_text:
  allText += one10K + " "

# generate word cloud from the string
wordcloud = WordCloud().generate(allText)
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off") 